In [1]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *
from delta import *


spark=SparkSession.builder.master("local[1]")\
        .appName("movie_local")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.4.4/jars/mysql-connector-j-9.1.0.jar" ) \
        .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/repository/database/") \
        .config("spark.sql.shuffle.partitions","8")\
        .enableHiveSupport()\
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark


25/02/13 22:02:49 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.15.9 instead (on interface en0)
25/02/13 22:02:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/13 22:02:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.4
PySpark Version :3.5.4


In [ ]:
! ls -l /Users/eduardoalberto/LoadFile/


In [2]:
df = spark.read.option("delimiter",';')\
               .option("header", "True")\
               .option("inferSchema", "True")\
               .csv("/Users/eduardoalberto/LoadFile/part-00000-055103f0-b275-4e27-b667-0c2c25d0636a-c000.csv")
df.printSchema()

root
 |-- id01: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- path_poster: string (nullable = true)
 |-- dt_release: date (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- runtime: double (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: boolean (nullable = true)
 |-- vote_average: integer (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- id02: integer (nullable = true)
 |-- name_geners: string (nullable = true)



In [32]:
df.toPandas()

,id01,id,name,poster_path,backdrop_path,budget,imdb_id,original_language,original_title,overview,...,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id02,name_geners
0,28,NaN,None,None,None,31500000,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",...,89460381,153.0,Released,This is the end...,Apocalypse Now,False,8,2112,28,Action
1,28,NaN,None,None,None,31500000,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",...,89460381,153.0,Released,This is the end...,Apocalypse Now,False,8,2112,28,Action
2,28,NaN,None,None,None,31500000,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",...,89460381,153.0,Released,This is the end...,Apocalypse Now,False,8,2112,28,Action
3,28,NaN,None,None,None,31500000,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",...,89460381,153.0,Released,This is the end...,Apocalypse Now,False,8,2112,28,Action
4,28,NaN,None,None,None,31500000,tt0078788,en,Apocalypse Now,"At the height of the Vietnam war, Captain Benj...",...,89460381,153.0,Released,This is the end...,Apocalypse Now,False,8,2112,28,Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40116,10749,NaN,None,None,None,0,tt0329632,de,Sophiiiie!,The film starts in the early evening of a norm...,...,0,107.0,Released,None,Sophiiiie!,False,6,1,10749,Romance
40117,10749,NaN,None,None,None,0,tt0329632,de,Sophiiiie!,The film starts in the early evening of a norm...,...,0,107.0,Released,None,Sophiiiie!,False,6,1,10749,Romance
40118,10749,NaN,None,None,None,0,tt0329632,de,Sophiiiie!,The film starts in the early evening of a norm...,...,0,107.0,Released,None,Sophiiiie!,False,6,1,10749,Romance
40119,10749,NaN,None,None,None,0,tt0329632,de,Sophiiiie!,The film starts in the early evening of a norm...,...,0,107.0,Released,None,Sophiiiie!,False,6,1,10749,Romance


In [ ]:
df01 = df.groupBy("name","imdb_id","overview","revenue","runtime","status","title","vote_average","vote_count","popularity","name_geners")\
         .agg(F.count("title").alias("total"))

df01.write.mode("overwrite").saveAsTable("dbdev150.tb_ratingMovie")



In [36]:
spark.table("dbdev150.tb_ratingMovie").orderBy(("total")).toPandas()

,name,imdb_id,overview,revenue,runtime,status,title,vote_average,vote_count,popularity,name_geners,total
0,The Sheriff and the Satellite Kid Collection,tt0079859,A young humanoid alien who gets stranded on ea...,0,88.0,Released,The Sheriff and the Satellite Kid,5,69,3.812074,TV Movie,390
1,Deuce Bigalow Collection,tt0205000,"Deuce Bigalow is a less than attractive, down ...",65535067,88.0,Released,Deuce Bigalow: Male Gigolo,5,314,6.567794,Music,487
2,In China They Eat Dogs Collection,tt0246692,"The last wish of the dying ""Monk"" is for his f...",0,95.0,Released,Old Men in New Cars: In China They Eat Dogs II,6,37,4.003761,Family,524
3,None,tt0268437,A lawyer becomes a fisherman from frustration....,0,97.0,Released,The Man Who Sued God,6,23,1.741405,Mystery,554
4,None,tt0036818,Die Feuerzangenbowle from Director Helmut Weis...,0,97.0,Released,The Punch Bowl,6,41,2.098595,Science Fiction,646
5,None,tt0169547,"Lester Burnham, a depressed suburban father in...",356296601,122.0,Released,American Beauty,7,3438,20.726578,Fantasy,704
6,Heart of Gold Collection,tt0168629,"Selma, a Czech immigrant on the verge of blind...",40031879,140.0,Released,Dancer in the Dark,7,392,10.684806,Animation,1124
7,None,tt0329632,The film starts in the early evening of a norm...,0,107.0,Released,Sophiiiie!,6,1,0.216899,Romance,1191
8,Finding Nemo Collection,tt0266543,"Nemo, an adventurous young clownfish, is unexp...",940335536,100.0,Released,Finding Nemo,7,6292,25.497794,Adventure,1514
9,Before... Collection,tt0381681,Nine years ago two strangers met by chance and...,15992615,80.0,Released,Before Sunset,7,734,7.048957,Crime,1682


### MERGE

In [ ]:
merge = """ MERGE INTO dbdev150.tb_ratingMovie
            USING 
"""